# 🏈 Team Lineup Optimizer

Generates optimal starting lineups for each team in the league based on:
- Player projections (mu, sigma, var, n_sources)
- Roster composition and bye weeks
- Benchmark waiver wire replacements

## Roster Configuration
- 1 QB
- 2 RB
- 2 WR
- 1 TE
- 1 FLEX (WR/RB/TE)
- 1 K
- 1 DST

## Benchmark Replacements
Players below these thresholds are replaced with "Waiver Pickup":
- QB: 18th ranked
- RB: 45th ranked
- WR: 55th ranked
- TE: 18th ranked
- K: 12th ranked
- DEF: 12th ranked


## 📦 Setup


In [1]:
import sqlite3
import sys
import pandas as pd
import json
import ast
from typing import List, Dict, Tuple
import os
from pathlib import Path
from dotenv import load_dotenv

# Add parent directory to path for imports
sys.path.append(str(Path().absolute().parent.parent))

# Load environment variables from root
load_dotenv(Path().absolute().parent.parent / '.env')

# Configuration
LEAGUE_ID = os.getenv("LEAGUE_ID", "1226433368405585920")
SEASON = "2025"
CURRENT_WEEK = 12

# Benchmark thresholds (rank at which we consider waiver wire replacement)
BENCHMARKS = {
    'QB': 18,
    'RB': 40,
    'WR': 50,
    'TE': 22,
    'K': 18,
    'DEF': 18  # Will map to D/ST in projections
}

# Roster slots
ROSTER_SLOTS = {
    'QB': 1,
    'RB': 2,
    'WR': 2,
    'TE': 1,
    'FLEX': 1,  # WR/RB/TE
    'K': 1,
    'DEF': 1
}

# Injury statuses that exclude players from lineups
EXCLUDED_INJURY_STATUSES = ['Out', 'IR', 'PUP', 'Suspended', 'Doubtful']

print("✓ Configuration loaded")
print(f"League ID: {LEAGUE_ID}")
print(f"Season: {SEASON}, Week: {CURRENT_WEEK}")

# Get absolute paths to databases
NOTEBOOK_DIR = Path().absolute()
BACKEND_DIR = NOTEBOOK_DIR.parent
DB_PROJ_PATH = str(BACKEND_DIR / "data" / "databases" / "projections.db")
DB_LEAGUE_PATH = str(BACKEND_DIR / "data" / "databases" / "league.db")


✓ Configuration loaded
League ID: 1226433368405585920
Season: 2025, Week: 12


## 📊 Load Data


In [2]:
# Connect to databases
conn_league = sqlite3.connect(DB_LEAGUE_PATH)
conn_proj = sqlite3.connect(DB_PROJ_PATH)

# Load rosters
query_rosters = f"""
    SELECT 
        r.roster_id,
        r.league_id,
        r.owner_id,
        r.team_name,
        r.players,
        r.wins,
        r.losses,
        r.ties,
        r.fpts,
        u.display_name,
        u.username
    FROM rosters r
    LEFT JOIN users u ON r.owner_id = u.user_id
    WHERE r.league_id = '{LEAGUE_ID}'
    ORDER BY r.roster_id
"""
df_rosters = pd.read_sql_query(query_rosters, conn_league)

print(f"✓ Loaded {len(df_rosters)} teams")
print(f"\nTeams:")
for _, row in df_rosters.iterrows():
    team_name = row['team_name'] or f"Team {row['roster_id']}"
    owner = row['display_name'] or row['username'] or 'Unknown'
    record = f"{row['wins']}-{row['losses']}"
    print(f"  {row['roster_id']:2}. {team_name} ({owner}) - {record}")


✓ Loaded 12 teams

Teams:
   1. Team 1 (xavierking4) - 8-3
   2. Team 2 (asadrafique) - 7-4
   3. Team 3 (amir812) - 6-5
   4. Team 4 (umarrahman30) - 1-10
   5. Team 5 (TBK41) - 6-5
   6. Team 6 (Jibraan) - 5-6
   7. Team 7 (mehdidrissi) - 4-7
   8. Team 8 (sahirsyed30) - 7-4
   9. Team 9 (Bilal879) - 3-8
  10. Team 10 (monkeyman966699696) - 6-5
  11. Team 11 (Ammady) - 6-5
  12. Team 12 (sfaizi24) - 7-4


In [3]:
# Load player projections for current week
query_projections = f"""
    SELECT 
        sleeper_player_id,
        player_name,
        position,
        week,
        mu,
        sigma,
        var,
        n_sources
    FROM player_week_stats
    WHERE week = {CURRENT_WEEK}
"""
df_projections = pd.read_sql_query(query_projections, conn_proj)

# Map DST to DEF for consistency (Sleeper stores defenses as DST)
df_projections['position'] = df_projections['position'].replace('DST', 'DEF')

print(f"✓ Loaded projections for {len(df_projections)} players")
print(f"\nTop 5 projected players:")
print(df_projections.nlargest(5, 'mu')[['player_name', 'position', 'mu', 'sigma']].to_string(index=False))


✓ Loaded projections for 537 players

Top 5 projected players:
        player_name position      mu     sigma
Christian McCaffrey       RB 26.6850 10.274155
         Drake Maye       QB 24.0700  7.528152
      Lamar Jackson       QB 23.0750  7.831347
         Josh Allen       QB 22.0400  8.997570
        Jalen Hurts       QB 22.0275  8.930926


In [4]:
# Load bye weeks for current week
query_byes = f"""
    SELECT DISTINCT team
    FROM nfl_schedules
    WHERE season = '{SEASON}' AND week = {CURRENT_WEEK} AND is_bye = 1
"""
df_byes = pd.read_sql_query(query_byes, conn_league)
bye_teams = set(df_byes['team'].tolist())

print(f"✓ Teams on bye in Week {CURRENT_WEEK}: {', '.join(sorted(bye_teams)) if bye_teams else 'None'}")


✓ Teams on bye in Week 12: DEN, LAC, MIA, WAS


In [5]:
# Load NFL players to get team affiliations and injury status
query_nfl_players = """
    SELECT 
        player_id,
        full_name,
        team,
        position,
        injury_status
    FROM nfl_players
"""
df_nfl_players = pd.read_sql_query(query_nfl_players, conn_league)

# Create lookup dictionaries
player_team_map = dict(zip(df_nfl_players['player_id'], df_nfl_players['team']))
player_injury_map = dict(zip(df_nfl_players['player_id'], df_nfl_players['injury_status']))

print(f"✓ Loaded {len(df_nfl_players)} NFL players")
print(f"✓ Injury statuses loaded")


✓ Loaded 3968 NFL players
✓ Injury statuses loaded


## 🎯 Calculate Benchmark Values

Determine the projected points for benchmark players at each position.


In [6]:
# Calculate benchmark values (the mu value at the benchmark rank for each position)
benchmark_values = {}

print("Calculating benchmark replacement values...\n")
print(f"{'Position':<8} {'Rank':<6} {'Player':<25} {'Mu':>8} {'Sigma':>8}")
print("-" * 60)

for position, rank in BENCHMARKS.items():
    pos_players = df_projections[df_projections['position'] == position].copy()
    pos_players = pos_players.sort_values('mu', ascending=False).reset_index(drop=True)
    
    if len(pos_players) >= rank:
        benchmark_player = pos_players.iloc[rank - 1]  # rank is 1-indexed
        benchmark_values[position] = {
            'mu': benchmark_player['mu'],
            'sigma': benchmark_player['sigma'],
            'var': benchmark_player['var'],
            'n_sources': benchmark_player['n_sources'],
            'player_name': benchmark_player['player_name']
        }
        print(f"{position:<8} {rank:<6} {benchmark_player['player_name']:<25} {benchmark_player['mu']:>8.2f} {benchmark_player['sigma']:>8.2f}")
    else:
        # Not enough players, use the lowest available
        if len(pos_players) > 0:
            benchmark_player = pos_players.iloc[-1]
            benchmark_values[position] = {
                'mu': benchmark_player['mu'],
                'sigma': benchmark_player['sigma'],
                'var': benchmark_player['var'],
                'n_sources': benchmark_player['n_sources'],
                'player_name': benchmark_player['player_name']
            }
            print(f"{position:<8} {len(pos_players):<6} {benchmark_player['player_name']:<25} {benchmark_player['mu']:>8.2f} {benchmark_player['sigma']:>8.2f} (Only {len(pos_players)} available)")
        else:
            benchmark_values[position] = {
                'mu': 0.0,
                'sigma': 0.0,
                'var': 0.0,
                'n_sources': 0,
                'player_name': 'No players available'
            }
            print(f"{position:<8} {rank:<6} {'No players available':<25} {0.0:>8.2f} {0.0:>8.2f}")

print("\n✓ Benchmark values calculated")


Calculating benchmark replacement values...

Position Rank   Player                          Mu    Sigma
------------------------------------------------------------
QB       18     Joe Flacco                   16.16     7.41
RB       40     Blake Corum                   6.93     9.15
WR       50     Tez Johnson                   7.72    10.89
TE       22     Brock Wright                  7.35     8.48
K        18     Daniel Carlson                7.20     5.09
DEF      18     KC Defense                    5.20     7.91

✓ Benchmark values calculated


## 🔧 Helper Functions


In [7]:
def parse_roster_players(players_str):
    """Parse the players column from roster data."""
    if pd.isna(players_str) or players_str == '':
        return []
    try:
        if isinstance(players_str, str):
            return ast.literal_eval(players_str)
        return players_str
    except:
        return []

def is_on_bye(player_id, player_team_map, bye_teams):
    """Check if a player is on bye this week."""
    team = player_team_map.get(player_id, None)
    return team in bye_teams if team else False

def is_injured_out(player_id, player_injury_map):
    """Check if a player has an injury status that excludes them from lineup."""
    injury_status = player_injury_map.get(player_id, None)
    if injury_status is None:
        return False
    return injury_status in EXCLUDED_INJURY_STATUSES

def get_player_projection(player_id, df_projections):
    """Get projection for a specific player."""
    player_proj = df_projections[df_projections['sleeper_player_id'] == player_id]
    if len(player_proj) > 0:
        return player_proj.iloc[0]
    return None

print("✓ Helper functions defined")


✓ Helper functions defined


In [8]:
def optimize_lineup(roster_players, df_projections, player_team_map, bye_teams, player_injury_map, benchmark_values):
    """
    Optimize lineup for a team's roster.
    Returns a list of lineup slots with player info.
    """
    # Get projections for all roster players
    roster_data = []
    for player_id in roster_players:
        proj = get_player_projection(player_id, df_projections)
        if proj is not None:
            # Skip players who are injured out
            if is_injured_out(player_id, player_injury_map):
                continue
            
            on_bye = is_on_bye(player_id, player_team_map, bye_teams)
            roster_data.append({
                'player_id': player_id,
                'player_name': proj['player_name'],
                'position': proj['position'],
                'mu': proj['mu'] if not on_bye else 0.0,  # 0 points if on bye
                'sigma': proj['sigma'],
                'var': proj['var'],
                'n_sources': proj['n_sources'],
                'on_bye': on_bye
            })
    
    # Sort by mu (descending)
    roster_data.sort(key=lambda x: x['mu'], reverse=True)
    
    # Initialize lineup
    lineup = []
    used_players = set()
    
    # Fill required positions
    for position in ['QB', 'K', 'DEF']:
        count = ROSTER_SLOTS.get(position, 0)
        pos_players = [p for p in roster_data if p['position'] == position and p['player_id'] not in used_players]
        
        for i in range(count):
            if i < len(pos_players):
                player = pos_players[i]
                # Check if below benchmark
                if player['mu'] < benchmark_values[position]['mu'] and not player['on_bye']:
                    # Replace with waiver pickup
                    lineup.append({
                        'slot': position,
                        'player_name': 'Waiver Pickup',
                        'position': position,
                        'mu': benchmark_values[position]['mu'],
                        'sigma': benchmark_values[position]['sigma'],
                        'var': benchmark_values[position]['var'],
                        'n_sources': benchmark_values[position]['n_sources'],
                        'is_replacement': True
                    })
                else:
                    lineup.append({
                        'slot': position,
                        'player_name': player['player_name'],
                        'position': player['position'],
                        'mu': player['mu'],
                        'sigma': player['sigma'],
                        'var': player['var'],
                        'n_sources': player['n_sources'],
                        'is_replacement': False
                    })
                    used_players.add(player['player_id'])
            else:
                # No player available, use waiver pickup
                lineup.append({
                    'slot': position,
                    'player_name': 'Waiver Pickup',
                    'position': position,
                    'mu': benchmark_values[position]['mu'],
                    'sigma': benchmark_values[position]['sigma'],
                    'var': benchmark_values[position]['var'],
                    'n_sources': benchmark_values[position]['n_sources'],
                    'is_replacement': True
                })
    
     # Fill RB, WR, TE positions
    for position in ['RB', 'WR', 'TE']:
        count = ROSTER_SLOTS.get(position, 0)
        pos_players = [p for p in roster_data if p['position'] == position and p['player_id'] not in used_players]
        
        for i in range(count):
            # Create numbered slot (e.g., RB1, RB2, WR1, WR2, TE1)
            slot_name = f'{position}{i+1}' if count > 1 else position
            if i < len(pos_players):
                player = pos_players[i]
                # Check if below benchmark
                if player['mu'] < benchmark_values[position]['mu'] and not player['on_bye']:
                    # Replace with waiver pickup
                    lineup.append({
                        'slot': slot_name,
                        'player_name': 'Waiver Pickup',
                        'position': position,
                        'mu': benchmark_values[position]['mu'],
                        'sigma': benchmark_values[position]['sigma'],
                        'var': benchmark_values[position]['var'],
                        'n_sources': benchmark_values[position]['n_sources'],
                        'is_replacement': True
                    })
                else:
                    lineup.append({
                        'slot': slot_name,
                        'player_name': player['player_name'],
                        'position': player['position'],
                        'mu': player['mu'],
                        'sigma': player['sigma'],
                        'var': player['var'],
                        'n_sources': player['n_sources'],
                        'is_replacement': False
                    })
                    used_players.add(player['player_id'])
            else:
                # No player available, use waiver pickup
                lineup.append({
                    'slot': slot_name,
                    'player_name': 'Waiver Pickup',
                    'position': position,
                    'mu': benchmark_values[position]['mu'],
                    'sigma': benchmark_values[position]['sigma'],
                    'var': benchmark_values[position]['var'],
                    'n_sources': benchmark_values[position]['n_sources'],
                    'is_replacement': True
                })
    
    # Fill FLEX (best remaining RB/WR/TE)
    flex_candidates = [p for p in roster_data 
                      if p['position'] in ['RB', 'WR', 'TE'] 
                      and p['player_id'] not in used_players]
    
    if len(flex_candidates) > 0:
        player = flex_candidates[0]  # Already sorted by mu
        # Check if below benchmark for their position
        if player['mu'] < benchmark_values[player['position']]['mu'] and not player['on_bye']:
            # Use best benchmark among eligible positions
            best_benchmark_pos = max(['RB', 'WR', 'TE'], 
                                    key=lambda p: benchmark_values[p]['mu'])
            lineup.append({
                'slot': 'FLEX',
                'player_name': 'Waiver Pickup',
                'position': best_benchmark_pos,
                'mu': benchmark_values[best_benchmark_pos]['mu'],
                'sigma': benchmark_values[best_benchmark_pos]['sigma'],
                'var': benchmark_values[best_benchmark_pos]['var'],
                'n_sources': benchmark_values[best_benchmark_pos]['n_sources'],
                'is_replacement': True
            })
        else:
            lineup.append({
                'slot': 'FLEX',
                'player_name': player['player_name'],
                'position': player['position'],
                'mu': player['mu'],
                'sigma': player['sigma'],
                'var': player['var'],
                'n_sources': player['n_sources'],
                'is_replacement': False
            })
            used_players.add(player['player_id'])
    else:
        # No flex candidate, use best benchmark
        best_benchmark_pos = max(['RB', 'WR', 'TE'], 
                                key=lambda p: benchmark_values[p]['mu'])
        lineup.append({
            'slot': 'FLEX',
            'player_name': 'Waiver Pickup',
            'position': best_benchmark_pos,
            'mu': benchmark_values[best_benchmark_pos]['mu'],
            'sigma': benchmark_values[best_benchmark_pos]['sigma'],
            'var': benchmark_values[best_benchmark_pos]['var'],
            'n_sources': benchmark_values[best_benchmark_pos]['n_sources'],
            'is_replacement': True
        })
    
    return lineup

print("✓ Lineup optimization function defined")


✓ Lineup optimization function defined


## 🎮 Generate Optimal Lineups for All Teams


In [9]:
# Generate optimal lineups for all teams
all_lineups = []

print("Generating optimal lineups...\n")

for _, roster_row in df_rosters.iterrows():
    roster_id = roster_row['roster_id']
    team_name = roster_row['team_name'] or f"Team {roster_id}"
    owner = roster_row['display_name'] or roster_row['username'] or 'Unknown'
    
    # Parse roster players
    roster_players = parse_roster_players(roster_row['players'])
    
    print(f"Team {roster_id}: {team_name} ({owner})")
    print(f"  Roster size: {len(roster_players)} players")
    
    # Optimize lineup
    lineup = optimize_lineup(roster_players, df_projections, player_team_map, bye_teams, player_injury_map, benchmark_values)
    
    # Add to results
    for slot_data in lineup:
        all_lineups.append({
            'roster_id': roster_id,
            'team_name': team_name,
            'owner': owner,
            'record': f"{roster_row['wins']}-{roster_row['losses']}",
            'slot': slot_data['slot'],
            'player_name': slot_data['player_name'],
            'position': slot_data['position'],
            'mu': slot_data['mu'],
            'sigma': slot_data['sigma'],
            'var': slot_data['var'],
            'n_sources': slot_data['n_sources'],
            'is_replacement': slot_data['is_replacement']
        })
    
    total_mu = sum(s['mu'] for s in lineup)
    print(f"  Projected total: {total_mu:.2f} points")
    print()

# Create DataFrame
df_lineups = pd.DataFrame(all_lineups)

print(f"✓ Generated lineups for {len(df_rosters)} teams")
print(f"✓ Total lineup slots: {len(df_lineups)}")


Generating optimal lineups...

Team 1: Team 1 (xavierking4)
  Roster size: 15 players
  Projected total: 119.85 points

Team 2: Team 2 (asadrafique)
  Roster size: 15 players
  Projected total: 107.74 points

Team 3: Team 3 (amir812)
  Roster size: 14 players
  Projected total: 126.61 points

Team 4: Team 4 (umarrahman30)
  Roster size: 15 players
  Projected total: 112.78 points

Team 5: Team 5 (TBK41)
  Roster size: 14 players
  Projected total: 120.62 points

Team 6: Team 6 (Jibraan)
  Roster size: 14 players
  Projected total: 117.72 points

Team 7: Team 7 (mehdidrissi)
  Roster size: 14 players
  Projected total: 118.31 points

Team 8: Team 8 (sahirsyed30)
  Roster size: 15 players
  Projected total: 121.95 points

Team 9: Team 9 (Bilal879)
  Roster size: 15 players
  Projected total: 110.87 points

Team 10: Team 10 (monkeyman966699696)
  Roster size: 15 players
  Projected total: 126.88 points

Team 11: Team 11 (Ammady)
  Roster size: 14 players
  Projected total: 118.18 points



## 📋 Create Projections Rosters Table

Export all owned players with their projections to league.db


## 🔧 Drop Old Projections Rosters Table

Drop the old table if it exists so it can be recreated with the updated schema


In [10]:
# Drop the old projections_rosters table if it exists
try:
    conn_league_drop = sqlite3.connect(DB_LEAGUE_PATH)
    cursor = conn_league_drop.cursor()
    cursor.execute("DROP TABLE IF EXISTS projections_rosters")
    conn_league_drop.commit()
    conn_league_drop.close()
    print("✓ Dropped old projections_rosters table (if it existed)")
except Exception as e:
    print(f"⚠ Error dropping table: {e}")

✓ Dropped old projections_rosters table (if it existed)


In [11]:
# Create projections_rosters table in league.db
print("Creating projections_rosters table in league.db...\n")

# Collect all owned players from all rosters
all_owned_players = []
for _, roster_row in df_rosters.iterrows():
    roster_id = roster_row['roster_id']
    team_name = roster_row['team_name'] or f"Team {roster_id}"
    roster_players = parse_roster_players(roster_row['players'])
    
    for player_id in roster_players:
        all_owned_players.append({
            'roster_id': roster_id,
            'team_name': team_name,
            'sleeper_player_id': player_id
        })

print(f"Found {len(all_owned_players)} total roster slots across all teams")

# Create DataFrame and join with player info and projections
df_owned = pd.DataFrame(all_owned_players)

# Join with nfl_players to get player details
df_owned = df_owned.merge(
    df_nfl_players[['player_id', 'full_name', 'position', 'team', 'injury_status']],
    left_on='sleeper_player_id',
    right_on='player_id',
    how='left'
)

# Split full_name into first and last name
df_owned['first_name'] = df_owned['full_name'].apply(lambda x: x.split(' ')[0] if pd.notna(x) and x else '')
df_owned['last_name'] = df_owned['full_name'].apply(lambda x: ' '.join(x.split(' ')[1:]) if pd.notna(x) and x and len(x.split(' ')) > 1 else '')

# Join with projections to get mu and var for current week
df_owned = df_owned.merge(
    df_projections[['sleeper_player_id', 'week', 'mu', 'var']],
    on='sleeper_player_id',
    how='left'
)

# Set week to CURRENT_WEEK for all players (even those without projections)
df_owned['week'] = CURRENT_WEEK

# Check if player is on bye or injured
df_owned['on_bye'] = df_owned['sleeper_player_id'].apply(
    lambda x: is_on_bye(x, player_team_map, bye_teams)
)
df_owned['is_injured'] = df_owned['sleeper_player_id'].apply(
    lambda x: is_injured_out(x, player_injury_map)
)

# Set mu and var to 0 for injured or bye players
df_owned.loc[df_owned['on_bye'] | df_owned['is_injured'], 'mu'] = 0.0
df_owned.loc[df_owned['on_bye'] | df_owned['is_injured'], 'var'] = 0.0

# Add season and timestamp
df_owned['season'] = SEASON
df_owned['timestamp'] = pd.Timestamp.now().isoformat()

# Rename NFL team column for clarity
df_owned = df_owned.rename(columns={'team': 'nfl_team'})

print(f"\nPlayers with projections: {df_owned['mu'].notna().sum()}")
print(f"Players without projections: {df_owned['mu'].isna().sum()}")
print(f"Players on bye: {df_owned['on_bye'].sum()}")
print(f"Players injured: {df_owned['is_injured'].sum()}")

# Calculate starting_status for each team
print("\nCalculating starting lineups...")
df_owned['starting_status'] = False

for roster_id in df_owned['roster_id'].unique():
    team_players = df_owned[df_owned['roster_id'] == roster_id].copy()
    team_players = team_players.sort_values('mu', ascending=False)
    
    starters = []
    
    # QB - highest mu
    qb_players = team_players[team_players['position'] == 'QB']
    if len(qb_players) > 0:
        starters.append(qb_players.iloc[0]['sleeper_player_id'])
    
    # K - highest mu
    k_players = team_players[team_players['position'] == 'K']
    if len(k_players) > 0:
        starters.append(k_players.iloc[0]['sleeper_player_id'])
    
    # DST/DEF - highest mu
    def_players = team_players[team_players['position'].isin(['DEF', 'DST'])]
    if len(def_players) > 0:
        starters.append(def_players.iloc[0]['sleeper_player_id'])
    
    # RB - top 2
    rb_players = team_players[team_players['position'] == 'RB']
    for i in range(min(2, len(rb_players))):
        starters.append(rb_players.iloc[i]['sleeper_player_id'])
    
    # WR - top 2
    wr_players = team_players[team_players['position'] == 'WR']
    for i in range(min(2, len(wr_players))):
        starters.append(wr_players.iloc[i]['sleeper_player_id'])
    
    # TE - top 1
    te_players = team_players[team_players['position'] == 'TE']
    if len(te_players) > 0:
        starters.append(te_players.iloc[0]['sleeper_player_id'])
    
    # FLEX - best remaining RB/WR/TE
    flex_candidates = team_players[
        team_players['position'].isin(['RB', 'WR', 'TE']) & 
        ~team_players['sleeper_player_id'].isin(starters)
    ]
    if len(flex_candidates) > 0:
        starters.append(flex_candidates.iloc[0]['sleeper_player_id'])
    
    # Mark starters as True
    df_owned.loc[
        (df_owned['roster_id'] == roster_id) & 
        (df_owned['sleeper_player_id'].isin(starters)),
        'starting_status'
    ] = True

print(f"Total starters marked: {df_owned['starting_status'].sum()}")

# Select final columns for the table
df_owned_final = df_owned[[
    'roster_id', 'team_name', 'sleeper_player_id', 'first_name', 'last_name', 
    'position', 'nfl_team', 'week', 'season', 'mu', 'var', 'starting_status', 'timestamp'
]]

# Export to league.db
try:
    conn_league_write = sqlite3.connect(DB_LEAGUE_PATH)
    cursor = conn_league_write.cursor()
    
    # Create table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS projections_rosters (
            roster_id INTEGER,
            team_name TEXT,
            sleeper_player_id TEXT,
            first_name TEXT,
            last_name TEXT,
            position TEXT,
            nfl_team TEXT,
            week INTEGER,
            season TEXT,
            mu REAL,
            var REAL,
            starting_status INTEGER,
            timestamp TEXT,
            PRIMARY KEY (sleeper_player_id, week, season)
        )
    """)
    
    print("\n✓ Table created/verified")
    
    # Insert records using INSERT OR REPLACE
    columns = ['roster_id', 'team_name', 'sleeper_player_id', 'first_name', 'last_name',
               'position', 'nfl_team', 'week', 'season', 'mu', 'var', 'starting_status', 'timestamp']
    
    for _, row in df_owned_final.iterrows():
        cursor.execute(f"""
            INSERT OR REPLACE INTO projections_rosters 
            ({', '.join(columns)}) 
            VALUES ({', '.join(['?'] * len(columns))})
        """, tuple(row[col] for col in columns))
    
    conn_league_write.commit()
    print(f"✓ Exported {len(df_owned_final)} roster players to league.db (projections_rosters table)")
    
    # Show sample of starters
    print("\nSample starters (top 10 by mu):")
    sample_query = f"""
        SELECT roster_id, team_name, first_name, last_name, position, nfl_team, mu, var, starting_status
        FROM projections_rosters
        WHERE week = {CURRENT_WEEK} AND season = '{SEASON}' and team_name = 'Team 12'
        ORDER BY mu DESC
        LIMIT 100
    """
    df_sample = pd.read_sql_query(sample_query, conn_league_write)
    print(df_sample.to_string(index=False))
    
    # Show summary stats
    print("\n\nStarting lineup summary:")
    summary_query = f"""
        SELECT 
            COUNT(*) as total_players,
            SUM(starting_status) as starters,
            SUM(CASE WHEN starting_status = 0 THEN 1 ELSE 0 END) as bench
        FROM projections_rosters
        WHERE week = {CURRENT_WEEK} AND season = '{SEASON}'
    """
    df_summary = pd.read_sql_query(summary_query, conn_league_write)
    print(df_summary.to_string(index=False))
    
    conn_league_write.close()
    print("\n✓ Database export complete")
    
except Exception as e:
    print(f"⚠ Database export error: {e}")
    import traceback
    traceback.print_exc()


Creating projections_rosters table in league.db...

Found 175 total roster slots across all teams

Players with projections: 175
Players without projections: 0
Players on bye: 23
Players injured: 18

Calculating starting lineups...
Total starters marked: 107

✓ Table created/verified
✓ Exported 175 roster players to league.db (projections_rosters table)

Sample starters (top 10 by mu):
 roster_id team_name first_name last_name position nfl_team        mu        var  starting_status
        12   Team 12       Puka     Nacua       WR      LAR 20.282500 102.108900                1
        12   Team 12        Tee   Higgins       WR      CIN 16.452500 102.009433                1
        12   Team 12    Derrick     Henry       RB      BAL 16.232500  83.840900                1
        12   Team 12      Tyler    Warren       TE      IND 12.992500  70.552900                1
        12   Team 12       Theo   Johnson       TE      NYG  9.830000  66.403733                1
        12   Team 12   

In [12]:
conn_league = sqlite3.connect(DB_LEAGUE_PATH)
conn_proj = sqlite3.connect(DB_PROJ_PATH)

summary_query = f"""
        SELECT *
        FROM projections_rosters
        WHERE 1=1
        --and week = {CURRENT_WEEK} 
        AND season = '{SEASON}' 
        and team_name = 'Team 8'
        ORDER BY mu DESC
        LIMIT 100
    """
df_summary = pd.read_sql_query(summary_query, conn_league)
df_summary

,roster_id,team_name,sleeper_player_id,first_name,last_name,position,nfl_team,week,season,mu,var,starting_status,timestamp
0,8,Team 8,6904,Jalen,Hurts,QB,PHI,12,2025,22.0275,79.761433,1,2025-11-20T12:05:30.991087
1,8,Team 8,8138,James,Cook,RB,BUF,12,2025,16.5900,99.225600,1,2025-11-20T12:05:30.991087
2,8,Team 8,8144,Chris,Olave,WR,NO,12,2025,16.1425,103.122233,1,2025-11-20T12:05:30.991087
3,8,Team 8,7525,DeVonta,Smith,WR,PHI,12,2025,14.5575,105.596900,1,2025-11-20T12:05:30.991087
4,8,Team 8,7049,Jauan,Jennings,WR,SF,12,2025,12.6750,104.010000,1,2025-11-20T12:05:30.991087
5,8,Team 8,1466,Travis,Kelce,TE,KC,12,2025,12.2425,64.834233,1,2025-11-20T12:05:30.991087
6,8,Team 8,9756,Jordan,Addison,WR,MIN,12,2025,10.9900,101.777600,0,2025-11-20T12:05:30.991087
7,8,Team 8,7567,Kenneth,Gainwell,RB,PIT,12,2025,10.1200,90.814400,1,2025-11-20T12:05:30.991087
8,8,Team 8,11533,Brandon,Aubrey,K,DAL,12,2025,9.6000,18.440000,1,2025-11-20T12:05:30.991087
9,8,Team 8,LV,LV,Defense,DST,LV,12,2025,8.0000,54.120000,1,2025-11-20T12:05:30.991087


In [13]:
# Display complete table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Format for display
df_display = df_lineups.copy()
df_display['mu'] = df_display['mu'].round(2)
df_display['sigma'] = df_display['sigma'].round(2)
df_display['var'] = df_display['var'].round(2)

# Mark replacements with text suffix
df_display['player_display'] = df_display.apply(
    lambda x: f"{x['player_name']} [WAIVER]" if x['is_replacement'] else x['player_name'], 
    axis=1
)

# Select columns for display
df_display_final = df_display[[
    'roster_id', 'team_name', 'owner', 'record', 'slot', 
    'player_display', 'position', 'mu', 'sigma', 'var', 'n_sources'
]].copy()

df_display_final.columns = [
    'Team ID', 'Team Name', 'Owner', 'Record', 'Slot',
    'Player', 'Pos', 'Mu', 'Sigma', 'Var', 'Sources'
]

print("="*120)
print("SIMULATED STARTING LINEUPS - All Teams")
print(f"Week {CURRENT_WEEK} Projections")
print("[WAIVER] = Waiver Pickup (Benchmark Replacement)")
print("="*120)
print()

print(df_display_final.to_string(index=False))

print()
print("="*120)


SIMULATED STARTING LINEUPS - All Teams
Week 12 Projections
[WAIVER] = Waiver Pickup (Benchmark Replacement)

 Team ID Team Name              Owner Record Slot                 Player Pos    Mu  Sigma    Var  Sources
       1    Team 1        xavierking4    8-3   QB       Matthew Stafford  QB 19.73   7.19  51.65        4
       1    Team 1        xavierking4    8-3    K         Evan McPherson   K  7.97   4.27  18.25        3
       1    Team 1        xavierking4    8-3  DEF             NE Defense DEF  7.10   8.04  64.68        2
       1    Team 1        xavierking4    8-3  RB1         Bijan Robinson  RB 21.34   9.07  82.35        4
       1    Team 1        xavierking4    8-3  RB2         Bhayshul Tuten  RB  7.12   9.28  86.17        4
       1    Team 1        xavierking4    8-3  WR1           Emeka Egbuka  WR 15.67  10.39 107.96        4
       1    Team 1        xavierking4    8-3  WR2       Jameson Williams  WR 13.51  10.05 100.92        4
       1    Team 1        xavierking4    8-

## 📈 Team Projections Summary


In [14]:
# Calculate team totals
team_totals = df_lineups.groupby(['roster_id', 'team_name', 'owner', 'record']).agg({
    'mu': 'sum',
    'sigma': lambda x: (sum(x**2))**0.5,  # Combined sigma (assuming independence)
    'var': 'sum',  # Variance is additive for independent variables
    'is_replacement': 'sum'  # Count replacements
}).reset_index()

team_totals.columns = ['Team ID', 'Team Name', 'Owner', 'Record', 'Total Mu', 'Combined Sigma', 'Total Var', 'Waiver Pickups']
team_totals = team_totals.sort_values('Total Mu', ascending=False)

team_totals['Total Mu'] = team_totals['Total Mu'].round(2)
team_totals['Combined Sigma'] = team_totals['Combined Sigma'].round(2)
team_totals['Total Var'] = team_totals['Total Var'].round(2)
team_totals['Waiver Pickups'] = team_totals['Waiver Pickups'].astype(int)

print("="*100)
print("TEAM PROJECTIONS SUMMARY")
print(f"Week {CURRENT_WEEK} - Ranked by Projected Points")
print("="*100)
print()

print(team_totals.to_string(index=False))

print()
print("="*100)
print(f"\nAverage projected points: {team_totals['Total Mu'].mean():.2f}")
print(f"Median projected points: {team_totals['Total Mu'].median():.2f}")
print(f"Highest projection: {team_totals['Total Mu'].max():.2f}")
print(f"Lowest projection: {team_totals['Total Mu'].min():.2f}")


TEAM PROJECTIONS SUMMARY
Week 12 - Ranked by Projected Points

 Team ID Team Name              Owner Record  Total Mu  Combined Sigma  Total Var  Waiver Pickups
      10   Team 10 monkeyman966699696    6-5    126.88           25.69     660.09               0
       3    Team 3            amir812    6-5    126.61           25.89     670.07               0
       8    Team 8        sahirsyed30    7-4    121.95           26.83     719.92               0
       5    Team 5              TBK41    6-5    120.62           26.60     707.42               0
       1    Team 1        xavierking4    8-3    119.85           26.11     681.80               0
      12   Team 12           sfaizi24    7-4    118.36           25.20     635.26               1
       7    Team 7        mehdidrissi    4-7    118.32           26.72     713.80               1
      11   Team 11             Ammady    6-5    118.18           26.28     690.83               1
       6    Team 6            Jibraan    5-6    117.72 

## 💾 Export Results


In [15]:
# Export to CSV
output_file = f'../data/csv/team_lineups_week_{CURRENT_WEEK}.csv'
df_lineups.to_csv(output_file, index=False)

summary_file = f'../data/csv/team_projections_summary_week_{CURRENT_WEEK}.csv'
team_totals.to_csv(summary_file, index=False)

print(f"✓ Exported detailed lineups to: {output_file}")
print(f"✓ Exported summary to: {summary_file}")

# Export to database
print("\nExporting to database...")

# Prepare lineup data for database (add week and timestamp)
df_lineups_db = df_lineups.copy()
df_lineups_db['week'] = CURRENT_WEEK
df_lineups_db['season'] = SEASON
df_lineups_db['timestamp'] = pd.Timestamp.now().isoformat()

# Prepare team summary data for database
team_totals_db = team_totals.copy()
team_totals_db['week'] = CURRENT_WEEK
team_totals_db['season'] = SEASON
team_totals_db['timestamp'] = pd.Timestamp.now().isoformat()

# Export to projections.db
try:
    conn_proj_write = sqlite3.connect(DB_PROJ_PATH)
    cursor = conn_proj_write.cursor()
    
    # Create team_lineups table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS team_lineups (
            roster_id INTEGER,
            team_name TEXT,
            owner TEXT,
            record TEXT,
            slot TEXT,
            player_name TEXT,
            position TEXT,
            mu REAL,
            sigma REAL,
            var REAL,
            n_sources INTEGER,
            is_replacement INTEGER,
            week INTEGER,
            season TEXT,
            timestamp TEXT,
            PRIMARY KEY (team_name, week, slot)
        )
    """)
    
    # Create team_projections_summary table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS team_projections_summary (
            roster_id INTEGER,
            team_name TEXT,
            owner TEXT,
            record TEXT,
            total_mu REAL,
            combined_sigma REAL,
            total_var REAL,
            waiver_pickups INTEGER,
            week INTEGER,
            season TEXT,
            timestamp TEXT,
            PRIMARY KEY (team_name, week)
        )
    """)
    
    print("✓ Tables created/verified")
    
    # Rename columns in team_totals_db to match database schema
    team_totals_db_renamed = team_totals_db.rename(columns={
        'Team ID': 'roster_id',
        'Team Name': 'team_name',
        'Owner': 'owner',
        'Record': 'record',
        'Total Mu': 'total_mu',
        'Combined Sigma': 'combined_sigma',
        'Total Var': 'total_var',
        'Waiver Pickups': 'waiver_pickups'
    })
    
    # Insert detailed lineups using INSERT OR REPLACE to handle duplicates
    # This will replace old records if they conflict with the primary key
    lineup_columns = ['roster_id', 'team_name', 'owner', 'record', 'slot', 'player_name', 
                      'position', 'mu', 'sigma', 'var', 'n_sources', 'is_replacement', 
                      'week', 'season', 'timestamp']
    
    for _, row in df_lineups_db.iterrows():
        cursor.execute(f"""
            INSERT OR REPLACE INTO team_lineups 
            ({', '.join(lineup_columns)}) 
            VALUES ({', '.join(['?'] * len(lineup_columns))})
        """, tuple(row[col] for col in lineup_columns))
    
    print(f"✓ Exported {len(df_lineups_db)} lineup slots to projections.db (team_lineups table)")
    
    # Insert team summary using INSERT OR REPLACE
    summary_columns = ['roster_id', 'team_name', 'owner', 'record', 'total_mu', 
                       'combined_sigma', 'total_var', 'waiver_pickups', 'week', 'season', 'timestamp']
    
    for _, row in team_totals_db_renamed.iterrows():
        cursor.execute(f"""
            INSERT OR REPLACE INTO team_projections_summary 
            ({', '.join(summary_columns)}) 
            VALUES ({', '.join(['?'] * len(summary_columns))})
        """, tuple(row[col] for col in summary_columns))
    
    print(f"✓ Exported {len(team_totals_db_renamed)} team summaries to projections.db (team_projections_summary table)")
    
    conn_proj_write.commit()
    conn_proj_write.close()
    
    print("✓ Database export complete")
    
except Exception as e:
    print(f"⚠ Database export error: {e}")
    print("  (CSV files were still created successfully)")
    import traceback
    traceback.print_exc()


✓ Exported detailed lineups to: ../data/csv/team_lineups_week_12.csv
✓ Exported summary to: ../data/csv/team_projections_summary_week_12.csv

Exporting to database...
✓ Tables created/verified
✓ Exported 108 lineup slots to projections.db (team_lineups table)
✓ Exported 12 team summaries to projections.db (team_projections_summary table)
✓ Database export complete


## 🔍 View Database Table

Query and display the raw team_lineups table from the database


In [16]:
# Query team_lineups table from database
conn_proj_view = sqlite3.connect(DB_PROJ_PATH)

# Query all records for current week
query = f"""
    SELECT *
    FROM team_lineups
    WHERE week = {CURRENT_WEEK} AND season = '{SEASON}'
    ORDER BY team_name, slot
"""

df_db_lineups = pd.read_sql_query(query, conn_proj_view)

print(f"Total records in team_lineups table for Week {CURRENT_WEEK}: {len(df_db_lineups)}")
print(f"\nColumns: {list(df_db_lineups.columns)}")
print(f"\n{'='*120}")
print("RAW DATABASE OUTPUT - team_lineups TABLE")
print(f"{'='*120}\n")

# Display all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)

print(df_db_lineups.to_string(index=False))

# Show summary by team
print(f"\n{'='*120}")
print("SUMMARY BY TEAM")
print(f"{'='*120}\n")
summary = df_db_lineups.groupby('team_name').agg({
    'mu': 'sum',
    'slot': 'count'
}).rename(columns={'slot': 'num_slots'})
summary = summary.sort_values('mu', ascending=False)
print(summary)

conn_proj_view.close()


Total records in team_lineups table for Week 12: 108

Columns: ['roster_id', 'team_name', 'owner', 'record', 'slot', 'player_name', 'position', 'mu', 'sigma', 'var', 'n_sources', 'is_replacement', 'week', 'season', 'timestamp']

RAW DATABASE OUTPUT - team_lineups TABLE

 roster_id team_name              owner record slot         player_name position        mu     sigma        var  n_sources  is_replacement  week season                  timestamp
         1    Team 1        xavierking4    8-3  DEF          NE Defense      DEF  7.100000  8.042388  64.680000          2               0    12   2025 2025-11-20T12:05:31.186068
         1    Team 1        xavierking4    8-3 FLEX          Josh Downs       WR  9.652500 10.098520 101.980100          4               0    12   2025 2025-11-20T12:05:31.186068
         1    Team 1        xavierking4    8-3    K      Evan McPherson        K  7.966667  4.272392  18.253333          3               0    12   2025 2025-11-20T12:05:31.186068
         1   

## 🔍 Individual Team Deep Dive

View detailed lineup for a specific team


In [17]:
def show_team_lineup(roster_id):
    """Display detailed lineup for a specific team."""
    team_data = df_lineups[df_lineups['roster_id'] == roster_id].copy()
    
    if len(team_data) == 0:
        print(f"Team {roster_id} not found")
        return
    
    team_name = team_data.iloc[0]['team_name']
    owner = team_data.iloc[0]['owner']
    record = team_data.iloc[0]['record']
    
    print("="*90)
    print(f"TEAM {roster_id}: {team_name}")
    print(f"Owner: {owner} | Record: {record}")
    print("="*90)
    print()
    
    print(f"{'Slot':<6} {'Player':<35} {'Pos':<5} {'Mu':>8} {'Sigma':>8} {'Var':>8} {'Src':>4}")
    print("-"*90)
    
    total_mu = 0
    total_var = 0
    
    for _, row in team_data.iterrows():
        marker = " [WAIVER]" if row['is_replacement'] else ""
        player_display = f"{row['player_name']}{marker}"
        print(f"{row['slot']:<6} {player_display:<35} {row['position']:<5} {row['mu']:>8.2f} {row['sigma']:>8.2f} {row['var']:>8.2f} {row['n_sources']:>4}")
        total_mu += row['mu']
        total_var += row['var']
    
    total_sigma = total_var ** 0.5
    
    print("-"*90)
    print(f"{'TOTAL':<6} {'':<35} {'':<5} {total_mu:>8.2f} {total_sigma:>8.2f} {total_var:>8.2f}")
    print()
    print("[WAIVER] = Waiver Pickup (Benchmark Replacement)")
    print("="*90)

# Example: Show lineup for team 1
show_team_lineup(8)


TEAM 8: Team 8
Owner: sahirsyed30 | Record: 7-4

Slot   Player                              Pos         Mu    Sigma      Var  Src
------------------------------------------------------------------------------------------
QB     Jalen Hurts                         QB       22.03     8.93    79.76    4
K      Brandon Aubrey                      K         9.60     4.29    18.44    3
DEF    LV Defense                          DEF       8.00     7.36    54.12    2
RB1    James Cook                          RB       16.59     9.96    99.23    4
RB2    Kenneth Gainwell                    RB       10.12     9.53    90.81    4
WR1    Chris Olave                         WR       16.14    10.15   103.12    4
WR2    DeVonta Smith                       WR       14.56    10.28   105.60    4
TE     Travis Kelce                        TE       12.24     8.05    64.83    4
FLEX   Jauan Jennings                      WR       12.67    10.20   104.01    4
--------------------------------------------------

## 🏆 Position Group Analysis


In [18]:
# Analyze by position group
position_analysis = df_lineups.groupby('slot').agg({
    'mu': ['mean', 'min', 'max', 'std'],
    'is_replacement': 'sum'
}).round(2)

position_analysis.columns = ['Avg Mu', 'Min Mu', 'Max Mu', 'Std Dev', 'Replacements']
position_analysis['Replacements'] = position_analysis['Replacements'].astype(int)

print("="*80)
print("POSITION GROUP ANALYSIS")
print(f"Across all {len(df_rosters)} teams")
print("="*80)
print()

print(position_analysis)

print()
print("="*80)


POSITION GROUP ANALYSIS
Across all 12 teams

      Avg Mu  Min Mu  Max Mu  Std Dev  Replacements
slot                                               
DEF     7.16    5.20    8.85     1.30             2
FLEX   10.82    8.04   13.72     1.80             0
K       8.39    7.20    9.60     0.88             1
QB     20.01   16.16   24.07     2.52             1
RB1    17.88   13.92   26.68     3.82             0
RB2    12.24    7.12   16.06     2.61             0
TE     11.79    7.74   17.76     2.85             0
WR1    16.82   11.47   21.04     2.61             0
WR2    13.21    9.88   16.45     2.13             0



## 🧹 Cleanup


In [19]:
# Close database connections
conn_league.close()
conn_proj.close()

print("✓ Database connections closed")


✓ Database connections closed
